In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")
include("mps_algorithms\\PUMPS_algorithms.jl")
include("mps_algorithms\\ES_preliminary.jl")




D=8;
chi=10;
W=10;
N=10;
kset=0:N-1;
EH_n=3;#number of entanglement spectrum per k point
Dtrun_method="svds";
Dtrun_init=200;
Dtrun_max=200;


J1=0.80902;
J2=0;
J3=0;
Jchi=0;
Jtrip=0.5878;

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);



In [2]:

    multi_threads=true;if Threads.nthreads()==1; multi_threads=false; end
    println("number of threads: "*string(Threads.nthreads()));flush(stdout);
    
    CTM_conv_tol=1e-6;
    trun_tol=1e-8;
    group_size=Int(round((10^8)/(chi*chi*W*W*D)));
    
    mpo_type="OO";#"O_O" or "OO", in my test "OO" is faster for large bond dimension
    
    pow=Int((N-2)/2);
    

    
    A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);
    
    filenm="LS_D_"*string(D)*"_chi_40.json"
    json_dict=read_json_state(filenm);
    
    bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);
    
    PEPS_tensor=bond_tensor;
    @tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
    A_unfused=PEPS_tensor;
    
    U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
    @tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

number of threads: 1


In [3]:
W=10;
CTM,U_L,U_D,U_R,U_U=try_CTM(D,chi,parameters, CTM_conv_tol, U_phy, A_unfused, A_fused);

O1=[];
O2=[];
Ag=[];

println("No itebd ground state found from saved data, now do itebd");flush(stdout);

Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];
@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];



load CTM from saved data directly


No itebd ground state found from saved data, now do itebd


In [4]:
unitcell_size=2;
for cc=1:unitcell_size-1
    U_unitcell=unitary(fuse(space(O1,4)⊗space(O1,4)),space(O1,4)⊗space(O1,4));
    @tensor O1[:]:=O1[-1,1,2,4]*O1[2,3,-3,5]*U_unitcell[-4,4,5]*U_unitcell'[1,3,-2];
    @tensor O2[:]:=O2[-1,1,2,4]*O2[2,3,-3,5]*U_unitcell[-4,4,5]*U_unitcell'[1,3,-2];
end

In [ ]:
# W=10;
# conv_check="singular_value"
# conv_tol=1e-6;
# CTM_init=[];
# CTM, OO=CTMRG_boundary_ground(O1,O2,W,conv_check,conv_tol,CTM_init)

In [5]:
W=10;


mps_virtual=SU₂Space(0=>1,1/2=>5);mps_phy=space(O1,2);
A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

#Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O",unitcell_size);


#save itebd data
O1_dict=convert(Dict,O1);
O2_dict=convert(Dict,O2);
Ag_dict=convert(Dict,Ag);
jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

#save initial CTM to compare with other codes
ES_filenm="Ag_mpo_tensors_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".mat";
matwrite(ES_filenm, Dict(
    "Ag" => convert(Array,Ag),
    "O1" => convert(Array,O1),
    "O2" => convert(Array,O2),
    "C1" => convert(Array,CTM["Cset"][1]),
    "C2" => convert(Array,CTM["Cset"][2]),
    "C3" => convert(Array,CTM["Cset"][3]),
    "C4" => convert(Array,CTM["Cset"][4]),
    "T1" => convert(Array,CTM["Tset"][1]),
    "T2" => convert(Array,CTM["Tset"][2]),
    "T3" => convert(Array,CTM["Tset"][3]),
    "T4" => convert(Array,CTM["Tset"][4])
); compress = false)



"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9998166056514061"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9998166056514103"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999982464447956"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999982464447954"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999999827365993"

...

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999999999999998"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.999999999999996"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999999999999951"

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999999999999999"

 16.707490 seconds (33.74 M allocations: 1.761 GiB, 3.74% gc time, 97.00% compilation time)
E=0.059438809662361355 - 0.0005345805866136211im, ov=0.25035856230108533
  1.304328 seconds (3.28 M allocations: 222.611 MiB, 66.50% compilation time)
E=0.060114874725802306 - 0.0005323061662811567im, ov=0.9889234156747395
  0.694381 seconds (1.12 M allocations: 98.177 MiB, 21.18% gc time)
E=0.060129981921004515 - 0.0005400987623794198im, ov=0.9995942721894905
  0.459087 seconds (1.12 M allocations: 98.079 MiB)
E=0.06015572933259057 - 0.0005422451224794081im, ov=0.9999152144489891
  0.473765 seconds (1.12 M allocations: 98.079 MiB)
E=0.0601839419447953 - 0.0005430484130951109im, ov=0.9999410566922141
  0.490016 seconds (1.12 M allocations: 98.079 MiB)
E=0.06020764288238094 - 0.0005434792240394154im, ov=0.9999586303487994
  1.179700 seconds (1.46 M allocations: 123.100 MiB, 12.03% gc time, 40.00% compilation time)
E=0.0005437603831509817 + 0.06022638261643251im, ov=0.999972116955751
  0.498785 se

In [ ]:
# Ag_,A_init_=ITEBD_boundary_groundstate(O2,O1,W,A_init,"OO");

# ES_filenm="Ag_mpo_tensors_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".mat";
# matwrite(ES_filenm, Dict(
#     "Ag" => convert(Array,Ag),
#     "Ag_" => convert(Array,Ag_),
#     "O1" => convert(Array,O1),
#     "O2" => convert(Array,O2),
#     "C1" => convert(Array,CTM["Cset"][1]),
#     "C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)

In [ ]:

    space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
    space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));
    
    AOA_sec=collect(sectors(space_AOA))
    AA_sec=collect(sectors(space_AA))
    
    @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
    U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
    @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    
    
    
    
    #normalize the MPO
    euL_set,_,_,_,_=FLR_eig(Ag,OO,20,space_AOA,AOA_sec);
    norm_coe=maximum(abs.(group_numbers(euL_set)));
    OO=OO/norm_coe;
    O1=O1/norm_coe;
    
    
    
    
    
    euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
    # println(euR_set)
    
    eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
    # println(eur_set)
    
    S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
    # println(S_set)
    
    s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
    # println(s_set)
    
    
    check_truncated_decomp_error=false;
    
    if mpo_type=="O_O"
        OO_transform=true;
    elseif mpo_type=="OO"
        OO_transform=false;
    end
    
    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
    euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,OO_transform,U_fuse_chichi)
    println("group information:");flush(stdout);
    println(DTrun_FLR_eig);flush(stdout);
    
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
    eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
    println("group information:");flush(stdout);
    println(Dtrun_GLR_eig);flush(stdout);
    
    
    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,OO_transform,U_fuse_chichi)
    println("group information:");flush(stdout);
    println(DTrun_FLR_svd);flush(stdout);
    
    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
    s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
    println("group information:");flush(stdout);
    println(Dtrun_GLR_svd);flush(stdout);
    
    
    
    
    ES_sectors=[0,1/2,1,3/2,2,5/2];
    
    #kset=0:0
    Eset=[];
    Trun_err=0;
    DTrun=0;
    println("calculate ES for N="*string(N));
    println("kset="*string(kset));flush(stdout);
    pow=round((N-2)/2);
    
    
    
    if Dtrun_method=="eigs"
        DTrun=length(group_numbers(SPIN_eig_set));
        println("DTrun="*string(DTrun));
    
        euRs=abs.(group_numbers(euR_set));
        Trun_err=(minimum(euRs)/maximum(euRs))^pow;
    
    
        euR_pow=deepcopy(euR_set_grouped);
        for ca=1:length(euR_pow)
            for cb=1:length(euR_pow[ca])
                euR_pow[ca][cb]=euR_pow[ca][cb]^Int(pow);
            end
        end
    
        kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,evR_set_grouped,euR_pow,evL_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig,mpo_type,multi_threads)
    
    elseif Dtrun_method=="svds"
        DTrun=length(group_numbers(SPIN_svd_set));
        println("DTrun="*string(DTrun));
    
        Ss=abs.(group_numbers(S_set));
        Trun_err=(minimum(Ss)/maximum(Ss));
    
        kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1,O2,OO,EH_n,N,kset,ES_sectors,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd,mpo_type,multi_threads)
    end
    
    ES_filenm="ES_"*Dtrun_method*"_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*"_N"*string(N)*"_kset"*string(kset[1])*"to"*string(kset[end])*".mat";
    matwrite(ES_filenm, Dict(
        "kset" => convert(Vector,kset),
        "ES_sectors" => ES_sectors,
        "Eset" => Eset,
        "Trun_err"=>Trun_err,
        "DTrun"=>DTrun
    ); compress = false)
    